# 目的
hstファイルを読み込み、その中身を確認する。

In [1]:
import os
import struct
import datetime

class HstFileAdapter(object):
    """ mt4 .hst file adapter """
    
    def __init__(self):
        """ 
        Initialization.
        The format of .hst file varies with its version.
        see <https://www.mql5.com/en/forum/149178>
        """
        self.headersize = {"400":148, "401":148}
        self.barsize = {"400":44, "401":60}
        self.cmd = {"400":['L', 'd', 'd', 'd', 'd', 'd'], 
                    "401":['Q', 'd', 'd', 'd', 'd', 'Q', 'L', 'Q']}
        self.byte = {"400":[4, 8, 8, 8, 8, 8], 
                     "401":[8, 8, 8, 8, 8, 8, 4, 8]}
        
        self.fxtf_start = datetime.datetime(1970, 1, 1, 0, 3, 0)
        
    def load(self, filepath, verbose=True):
        """ load a .hst file. """
        if not os.path.exists(filepath):
            raise FileNotFoundError
        
        self.filepath = filepath
        with open(filepath, 'rb') as fh:
            # read header
            self.ver = struct.unpack('L'  , fh.read( 4))[0]
            headersize = self.headersize[str(self.ver)]
            barsize = self.barsize[str(self.ver)]
            cmd  = self.cmd[str(self.ver)]
            byte = self.byte[str(self.ver)]
            
            self.copyright = ""
            for ii in range(64):
                self.copyright += str(struct.unpack("c", fh.read(1))[0].decode())
            self.symbol = ""
            for ii in range(12):
                self.symbol += str(struct.unpack("c", fh.read(1))[0].decode())
            self.period    = struct.unpack('L'  , fh.read( 4))[0]
            self.digits    = struct.unpack('L'  , fh.read( 4))[0]
            self.timesign  = struct.unpack('L'  , fh.read( 4))[0]
            self.last_sync = struct.unpack('L'  , fh.read( 4))[0]
            self.unused    = struct.unpack('13L', fh.read(52))[0]
            
            # calc data size
            filesize = os.path.getsize(filepath)
            datasize = filesize - headersize
            self.bars = int(datasize / barsize)
            
            if verbose:
                self.printHeader()
            
            # read data
            fh.seek(headersize)
            self.data = []
            for k in range(self.bars):
#                 for i, j in zip(cmd,byte):
#                     print(struct.unpack(i, fh.read(j))[0])
#                 lst = [struct.unpack(i, fh.read(j))[0] for i,j in zip(cmd,byte)]
                self.data.append([struct.unpack(i, fh.read(j))[0] for i,j in zip(cmd,byte)])
    
    def printHeader(self):
        """ view header data """
        print('ver       ', self.ver)
        print('copyright ', self.copyright)
        print('symbol    ', self.symbol)
        print('period    ', self.period)
        print('digits    ', self.digits)
        print('timesign  ', self.timesign)
        print('last_sync ', self.last_sync)
        print('unused    ', self.unused)
        print('----------')
        print('bars      ', self.bars)
    
    def printData(self):
        """ view history data """
        for i in self.data:
            print(i)
    
    def toCSV(self, filepath):
        pass
    
    def outputHstTest(self, outputfilename = "outputtest.hst"):
        """ .hst output test """
        with open(outputfilename,'wb') as fh:
            fh.write(struct.pack('L', self.ver))
            fh.write(struct.pack('64s', self.copyright))
            fh.write(struct.pack('12s', self.symbol))
            fh.write(struct.pack('L', self.period))
            fh.write(struct.pack('L', self.digits))
            fh.write(struct.pack('L', self.timesign))
            fh.write(struct.pack('L', self.last_sync))
            fh.write(struct.pack('52s', ""))
            cmd  = 'q4d2qi'
            for i, v in enumerate(self.data):
                fh.write(struct.pack(cmd, self.data[i][0], float(self.data[i][1]),
                        float(self.data[i][2]), float(self.data[i][3]),
                        float(self.data[i][4]), int(self.data[i][5]), 0, 0))
            
        return outputfilename

In [2]:
adapter = HstFileAdapter()

In [3]:
# filepath = "C:/Users/Surpris/Downloads/USDJPY43200.hst"
basepath = "C:/Users/Surpris/AppData/Roaming/MetaQuotes/Terminal/5510A4731F12807177E1BDF3126F1128/history/FXTrade-Demo FXTF/"
fname = "USDJPY-cd1 - コピー.hst"
filepath = basepath + fname
adapter.load(filepath)
adapter.data[-100:]

ver        401
copyright  (C)opyright 2003, MetaQuotes Software Corp.                     
symbol     USDJPY-cd   
period     1
digits     3
timesign   1496857013
last_sync  0
unused     0
----------
bars       544987


[[1501153860, 111.174, 111.198, 111.172, 111.195, 145, 0, 0],
 [1501153920, 111.196, 111.197, 111.181, 111.186, 121, 0, 0],
 [1501153980, 111.187, 111.21, 111.186, 111.209, 137, 0, 0],
 [1501154040, 111.208, 111.261, 111.208, 111.246, 192, 0, 0],
 [1501154100, 111.247, 111.285, 111.246, 111.279, 146, 0, 0],
 [1501154160, 111.28, 111.283, 111.273, 111.274, 97, 0, 0],
 [1501154220, 111.273, 111.283, 111.271, 111.279, 98, 0, 0],
 [1501154280, 111.28, 111.288, 111.276, 111.28, 130, 0, 0],
 [1501154340, 111.279, 111.286, 111.271, 111.274, 165, 0, 0],
 [1501154400, 111.275, 111.331, 111.274, 111.321, 177, 0, 0],
 [1501154460, 111.322, 111.333, 111.314, 111.316, 161, 0, 0],
 [1501154520, 111.315, 111.315, 111.285, 111.286, 155, 0, 0],
 [1501154580, 111.287, 111.291, 111.28, 111.285, 116, 0, 0],
 [1501154640, 111.286, 111.31, 111.286, 111.294, 114, 0, 0],
 [1501154700, 111.292, 111.297, 111.287, 111.294, 97, 0, 0],
 [1501154760, 111.295, 111.304, 111.288, 111.291, 100, 0, 0],
 [1501154820, 111

In [4]:
adapter2 = HstFileAdapter()
filepath = basepath + "USDJPY-cd1.hst"
adapter2.load(filepath)
adapter2.data[-100:]

ver        401
copyright  (C)opyright 2003, MetaQuotes Software Corp.                     
symbol     USDJPY-cd   
period     1
digits     3
timesign   1496857013
last_sync  0
unused     0
----------
bars       547840


[[1501501140, 110.637, 110.65, 110.63, 110.642, 92, 0, 0],
 [1501501200, 110.645, 110.651, 110.644, 110.648, 47, 0, 0],
 [1501501260, 110.649, 110.649, 110.617, 110.624, 93, 0, 0],
 [1501501320, 110.623, 110.625, 110.618, 110.618, 112, 0, 0],
 [1501501380, 110.621, 110.627, 110.62, 110.625, 82, 0, 0],
 [1501501440, 110.626, 110.639, 110.623, 110.639, 93, 0, 0],
 [1501501500, 110.64, 110.645, 110.614, 110.623, 77, 0, 0],
 [1501501560, 110.624, 110.634, 110.62, 110.62, 85, 0, 0],
 [1501501620, 110.621, 110.63, 110.618, 110.628, 119, 0, 0],
 [1501501680, 110.632, 110.639, 110.627, 110.631, 89, 0, 0],
 [1501501740, 110.629, 110.634, 110.627, 110.631, 46, 0, 0],
 [1501501800, 110.63, 110.636, 110.629, 110.634, 62, 0, 0],
 [1501501860, 110.633, 110.657, 110.627, 110.652, 92, 0, 0],
 [1501501920, 110.651, 110.664, 110.646, 110.663, 65, 0, 0],
 [1501501980, 110.662, 110.679, 110.652, 110.67, 70, 0, 0],
 [1501502040, 110.671, 110.672, 110.661, 110.664, 75, 0, 0],
 [1501502100, 110.663, 110.675,

In [ ]:
now = datetime.datetime(2017, 7, 27, 9, 17, 0)
delta = now - datetime.datetime(1970, 1, 1, 0, 3, 0) # start datetime of FXT?
delta.total_seconds()

In [ ]:
1501146900, "2017.07.27 9:18"

In [ ]:
delta.total_seconds() - 1501146900